### Data Cleaning ###

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [3]:
df_original = pd.read_csv(r'../data/2015.csv')

In [4]:
df_original.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01292015',b'01',b'29',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
1,1.0,1.0,b'01202015',b'01',b'20',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,2.0
2,1.0,1.0,b'02012015',b'02',b'01',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
3,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,9.0
4,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0


In [5]:
df_original.shape

(441456, 330)

# Feature Selection


Because our database is very large, we have reduced the number of features to obtain more understandable results when using simpler algorithms like the KNN. To do this, a brief research on the internet was done to identify the risk factors according to the opinion of the experts. The main sources were the United Kingdom National Health Service (NHS - [link](https://www.nhs.uk/conditions/cardiovascular-disease/)), the Center for Disease Control and Prevention (CDC - [link](https://www.cdc.gov/heartdisease/risk_factors.htm)) and the World Health Organization (WHO - [link](https://www.who.int/news-room/fact-sheets/detail/cardiovascular-diseases-(cvds))).

According to our researched, the following risks were selected:
1. **blood pressure (high)** code as _RFHYPE5
2. **cholesterol (high)** code as _TOLDHI2
3. **overweigh**t code as BMI5
4. **smoking** code as SMOKE100
5. **diabetes** code as DIABETE3
6. **fruit consumption** as _FRTLT1
7. **vegetables consuption** code as _VEGLT1
8. **alcohol consumption** code as _RFDRHV5 
9. **inactivity** code as _TOTINDA 
10. **age** code as _AGEG5YR
11. **gender** code as SEX
12. **targer heart disease** code as _MICHD

In [27]:
# select specific columns
df = df_original[[
        '_MICHD', 
        '_RFHYPE5',  
        'TOLDHI2',
        '_BMI5', 
        'SMOKE100', 
        'DIABETE3', 
        '_TOTINDA', 
        '_FRTLT1', 
        '_VEGLT1', 
        '_RFDRHV5', 
        'SEX', 
        '_AGEG5YR' ]]


**Rename columns to make more readable**

In [28]:
# rename columns
df = df.rename(columns = {
                '_MICHD':'HeartDiseaseorAttack', 
                '_RFHYPE5':'HighBP',  
                'TOLDHI2':'HighChol', 
                '_BMI5':'BMI', 
                'SMOKE100':'Smoker', 
                'DIABETE3':'Diabetes', 
                '_TOTINDA':'PhysActivity', 
                '_FRTLT1':'Fruits', 
                '_VEGLT1':"Veggies", 
                '_RFDRHV5':'HvyAlcoholConsump', 
                'SEX':'SexIsMale', 
                '_AGEG5YR':'AgeGroup'})
                
df.head()

,HeartDiseaseorAttack,HighBP,HighChol,BMI,Smoker,Diabetes,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,SexIsMale,AgeGroup
0,2.0,2.0,1.0,4018.0,1.0,3.0,2.0,2.0,1.0,1.0,2.0,9.0
1,2.0,1.0,2.0,2509.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,7.0
2,NaN,1.0,1.0,2204.0,NaN,3.0,9.0,9.0,9.0,9.0,2.0,11.0
3,2.0,2.0,1.0,2819.0,2.0,3.0,2.0,1.0,2.0,1.0,2.0,9.0
4,2.0,1.0,2.0,2437.0,2.0,3.0,2.0,9.0,1.0,1.0,2.0,9.0


# Handling individual features

In this section we will analyze each feature's labels following the information in the codebook. Since we still want to create a dataset with imputed data, we will modify both datasets df and df_droppedNaN

## Target: HeartDisease (_MICHD)

Respondents that have ever reported having coronary heart disease (CHD) or myocardial infarction (MI). 

**Labels:**
* 1: Reported have CHD or MI.
* 2: Did not report having CHD or MI.

For logic purposes, label 2 will be substitute by 0. 

In [29]:
# HeartDiseaseorAttack column
df['HeartDiseaseorAttack'] = df['HeartDiseaseorAttack'].replace({2:0})

## Feature: HighBP (_RFHYPE5)

Adults who have been told they have high blood pressure by a doctor, nurse, or other health professional. 

**Labels:**
* 1: No
* 2: Yes
* 9: Don't know

For logic purposes, label 1 will be substitute by 0 and label 2 by 1. 

In [30]:
# HighBP
df['HighBP'] = df['HighBP'].replace({9:np.nan})
df['HighBP'] = df['HighBP'].replace({1:0})
df['HighBP'] = df['HighBP'].replace({2:1})

## Feature: HighChol (TOLDHI2)

Adults who have been told by a doctor, nurse or other health professional that their blood cholesterol is high

**Labels:**
* 1: Yes
* 2: No
* 7: Don't know
* 9: Refused

For logic purposes, label 1 will be substitute by 0 and label 2 by 1. 

In [31]:
# HighChol
df['HighChol'] = df['HighChol'].replace({7:np.nan})
df['HighChol'] = df['HighChol'].replace({9:np.nan})
df['HighChol'] = df['HighChol'].replace({2:0})

## Feature BMI

Body Mass Index (BMI). 

In [10]:
# nothing to be done here

## Feature Smoker (SMOKE100) 


People that have smoked at least 100 cigarettes in their entire life.  

**Labels:**
* 1: yes
* 2: No
* 7: Don't know
* 9: refuse to answer

Label 2 will be substitute by 0 and label 7 and 9 will be deleted. 

In [32]:
# Smoker
df['Smoker'] = df['Smoker'].replace({7:np.nan})
df['Smoker'] = df['Smoker'].replace({9:np.nan})
df['Smoker'] = df['Smoker'].replace({2:0})

## Feature Diabetes (DIABETE3)


People that have been told to suffer from diabetes.  

**Labels:**
* 1: yes
* 2: yes but only during pregnancy
* 3: no
* 4: pre-diabetes or borderline diabetes. 
* 7: Don't know
* 9: refuse to answer

We will just consider yes(1) or no (3) answers. To simplify, the rest of the options will be deleted. 

In [33]:
# Diabetes
df['Diabetes'] = df['Diabetes'].replace({2:np.nan})
df['Diabetes'] = df['Diabetes'].replace({4:np.nan})
df['Diabetes'] = df['Diabetes'].replace({7:np.nan})
df['Diabetes'] = df['Diabetes'].replace({9:np.nan})
df['Diabetes'] = df['Diabetes'].replace({3:0})

## Feature: Physical Activity (_TOTINDA)


Adults who reported doing physical activity or exercise during the past 30days other than their regular job. 

**Labels:**
* 1: yes
* 2: no
* 9: don't know or refuse to answer. 

Label 2 will be substitute by 0 and label 9 will be deleted.  

In [34]:
# PhysActivity
df['PhysActivity'] = df['PhysActivity'].replace({2:0})
df['PhysActivity'] = df['PhysActivity'].replace({9:np.nan})

## Feature: Fruit consumption (_FRTLT1)

Consume Fruit 1 or more times per day. 


**Labels:**
* 1: yes
* 2: no
* 9: don't know or refuse to answer. 

Label 2 will be substitute by 0 and label 9 will be deleted.  

In [35]:
# Fruits
df['Fruits'] = df['Fruits'].replace({2:0})
df['Fruits'] = df['Fruits'].replace({9:np.nan})

## Feature: Veggies (_VEGLT1)

People consume vegetables 1 or more times per day. 


**Labels:**
* 1: yes
* 2: no
* 9: don't know or refuse to answer. 

Label 2 will be substitute by 0 and label 9 will be deleted.  

In [36]:
# Veggies
df['Veggies'] = df['Veggies'].replace({2:0})
df['Veggies'] = df['Veggies'].replace({9:np.nan})

## Feature: Alcohol consumption (_RFDRHV5)

Adult men having more than 14 drinks per week and adult women having more than 7 drinks per week. 

**Labels:**
* 1: no
* 2: yes
* 9: don't know or refuse to answer. 

Label 1 will be substitute by 0, label 2 by 1, and label 9 will be deleted.  

In [37]:
# HvyAlcoholConsump
df['HvyAlcoholConsump'] = df['HvyAlcoholConsump'].replace({1:0})
df['HvyAlcoholConsump'] = df['HvyAlcoholConsump'].replace({2:1})
df['HvyAlcoholConsump'] = df['HvyAlcoholConsump'].replace({9:np.nan})

## Feature: gender (sex)

Indicate the gender of respondent. 

**Labels:**
* 1: male
* 2: female

Label 2 will be substitute by 0.  

In [38]:
# SexIsMale
df['SexIsMale'] = df['SexIsMale'].replace({2:0})

## Feature: age group (_AGEG5YR)

There are 14- level age categories 

**Labels:**
* 1: 18 to 24 years
* 2: 25 to 29 years
* 3: 30 to 34 years
* 4: 35 to 39 years
* 5: 40 to 44 years
* 6: 45 to 49 years
* 7: 50 to 54 years
* 8: 55 to 59 years
* 9: 60 to 64 years
* 10: 65 to 69 years
* 11: 70 to 74 years
* 12: 75 to 79 years
* 13: 80 or older
* 14: don't know or refuse answering

Label 14 will be deleted. 

In [39]:
# AgeGroup
df['AgeGroup'] = df['AgeGroup'].replace({14:np.nan})

# Handling missing values

First we check how much percentage of the data is missing.

In [40]:
print(round((((df.isnull().sum()).sum() / np.product(df.shape)) * 100), 2))

5.62


We check then, which columns have the highest percentage of missing values. 

In [41]:
print(df.isnull().sum())

HeartDiseaseorAttack     3942
HighBP                   1367
HighChol                62715
BMI                     36398
Smoker                  17549
Diabetes                12096
PhysActivity            37992
Fruits                  43711
Veggies                 51117
HvyAlcoholConsump       25733
SexIsMale                   0
AgeGroup                 5336
dtype: int64


Missing values can be handle with one of the following options:

1. Invest more in data collection: This option is not possible for us due to time-limitations and out of scope responsabilities.
2. Data exclusion: Remove rows with missing data.
3. Data imputation: Replace values by artificial new data.
4. Just leave the missing values: Not recommended.

According to the slides from our class (Block05-06 page 25), we should never impute the target variable in supervised learning. Therefore, we have decided to delete all rows with a missing target value (HeartDisease). 

In [42]:
# We dropped all rows whose value for HeartDisease is Nan. Or better explain, we keep all the rows that have a value that is not NaN
df = df[df['HeartDiseaseorAttack'].notna()]
print(df.isnull().sum())

HeartDiseaseorAttack        0
HighBP                   1232
HighChol                61973
BMI                     35929
Smoker                  17257
Diabetes                11772
PhysActivity            37500
Fruits                  43018
Veggies                 50292
HvyAlcoholConsump       25340
SexIsMale                   0
AgeGroup                 5200
dtype: int64


In [43]:
df.shape

(437514, 12)

## Dataset with dropped NaN

In [44]:
df_droppedNaN = df.dropna()
print(df_droppedNaN.isnull().sum())

HeartDiseaseorAttack    0
HighBP                  0
HighChol                0
BMI                     0
Smoker                  0
Diabetes                0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
SexIsMale               0
AgeGroup                0
dtype: int64


In [45]:
print(df_droppedNaN.shape)
print(df.shape)

(292745, 12)
(437514, 12)


In [46]:
df_droppedNaN.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292745 entries, 0 to 441455
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   HeartDiseaseorAttack  292745 non-null  float64
 1   HighBP                292745 non-null  float64
 2   HighChol              292745 non-null  float64
 3   BMI                   292745 non-null  float64
 4   Smoker                292745 non-null  float64
 5   Diabetes              292745 non-null  float64
 6   PhysActivity          292745 non-null  float64
 7   Fruits                292745 non-null  float64
 8   Veggies               292745 non-null  float64
 9   HvyAlcoholConsump     292745 non-null  float64
 10  SexIsMale             292745 non-null  float64
 11  AgeGroup              292745 non-null  float64
dtypes: float64(12)
memory usage: 29.0 MB


In [47]:
# save as new csv
df_droppedNaN.to_csv('../data/2015_cleaned_droppedNaN.csv', index=False)


## Dataset with imputed NaN

In this section we created a dataset with imputed NaN for learning purposes. We would also like to compare the performance between both datasets. We are aware that imputation might introduce biases. 

**Scale Data**

A critical point is that the KNN Imputer is a distance-based imputation method and it requires us to normalize our data. Otherwise, the different scales of our data will lead the KNN Imputer to generate biased replacements for the missing values. For simplicity, we will use Scikit-Learn’s MinMaxScaler which will scale our variables to have values between 0 and 1.

Source: [link text](https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e)

In [ ]:
# Todo: CHECK IF SCALING IS NECESSARY BEFORE KNN IMPUTING

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# df_scaled = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

In [ ]:
# df.head()

,HighBP,HighChol,DIABETE3,Smoker,AlcoholConsump,PhysActivity,BMI,Age,Sex,HeartDisease
0,2.0,1.0,3.0,1.0,1.0,2.0,4018.0,9.0,2.0,2.0
1,1.0,2.0,3.0,1.0,1.0,1.0,2509.0,7.0,2.0,2.0
3,2.0,1.0,3.0,2.0,1.0,2.0,2819.0,9.0,2.0,2.0
4,1.0,2.0,3.0,2.0,1.0,2.0,2437.0,9.0,2.0,2.0
5,2.0,2.0,3.0,2.0,1.0,1.0,2652.0,11.0,2.0,2.0


In [ ]:
# df_scaled.head()

,HighBP,HighChol,DIABETE3,Smoker,AlcoholConsump,PhysActivity,BMI,Age,Sex,HeartDisease
0,0.125,0.000,0.25,0.000,0.0,0.125,0.320255,0.615385,1.0,1.0
1,0.000,0.125,0.25,0.000,0.0,0.000,0.148641,0.461538,1.0,1.0
2,0.125,0.000,0.25,0.125,0.0,0.125,0.183896,0.615385,1.0,1.0
3,0.000,0.125,0.25,0.125,0.0,0.125,0.140453,0.615385,1.0,1.0
4,0.125,0.125,0.25,0.125,0.0,0.000,0.164904,0.769231,1.0,1.0


Now we can create the KNN-Imputater 

In [ ]:
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean', add_indicator=False)
# df_imputed = imputer.fit_transform(df_scaled)

In [ ]:
# df_imputed = pd.DataFrame(df_imputed, columns=df_scaled.columns)
# df_imputed.to_csv('../2015_cleaned_imputedNaN.csv')